In [ ]:
import requests
#from requests import Request, Session
from requests_aws_sign import AWSV4Sign
import requests_cache
from boto3 import session
import os
from six import string_types
import logging as logger

#get running enviroment  - necessariy for URL
env =os.environ.get('ENV', "dev")


# Establish credentials using boto3
session = session.Session()
credentials = session.get_credentials()
region = session.region_name or 'eu-central-1'

#set urls for the two functions
uri_alias =  "https://app-{env}.sourcingbot.com/entity/manufacturer/alias/{name}"
uri_full =  "https://app-{env}.sourcingbot.com/entity/manufacturer/{name}"
headers={"Content-Type":"application/json"}
service = 'execute-api'
auth=AWSV4Sign(credentials, region, service)
requests_cache.install_cache(backend='memory')

In [ ]:


def get_alias(mfr):
    if isinstance(mfr, string_types):
        url = uri_alias.format_map({"name":mfr, "env":env})
        response= requests.get(url, auth=auth, headers=headers)
        if response.ok:
            logger.error('result from cache {0}'.format(response.from_cache))

            data = response.json()
            return data
        else:
            return False
    else:
        logger.error('alias is not a string')
        return False

def get_full(mfr):
    if isinstance(mfr, string_types):
        url = uri_full.format_map({"name":mfr, "env":env})
        response= requests.get(url, auth=auth, headers=headers)
        if response.ok:
            data = response.json()
            return data
        else:
            return False
    else:
        logger.error('alias is not a string')
        return False

def normalize_mfr(mfr):
    data = {"mfr": { "main":"","aliases":[]}}
    if isinstance(mfr, string_types):
        alias = get_alias((mfr))
        print(alias)
        if alias:
            if 'data' in alias and 'parent' in alias['data']:
                main = alias['data']['parent']
                data['mfr']['main'] = main
                alias2 =  get_full(main)
                if 'data' in alias2 and 'alias' in alias2['data']:
                    print(alias['data'])
                    data['mfr']['aliases'] = alias2['data']['alias']
            return data
        elif not alias:
            logger.error('mfr mapping not found')
            data['mfr']['main'] = mfr
            return data


            
        
    else:
        logger.error('bad mfr mapping received')
        return False


In [ ]:
print(normalize_mfr("wurth electronics"))


In [ ]:
from timeit import default_timer as timer

start = timer()
for i in range(30):
    get_alias("wurth electronics")
end = timer()
print(end - start)

print(get_full("wurth electronics"))

#normalize_mfr("hart")

In [ ]:
import timeit


In [ ]:
#timeit.timeit('normalize_mfr', number=100)
timeit.timeit('get_alias("pizza")', "from __main__ import get_alias",number=30)

#timeit.timeit('print(set([get_alias("test") for n in range(5)]))', "from __main__ import get_alias", number=1)


In [ ]:
timeit.timeit('"-".join(str(n) for n in range(99))', number=100000)


In [ ]:
#timeit.timeit('normalize_mfr("test")', number=100)
timeit.timeit('normalize_mfr("pizza")', "from __main__ import normalize_mfr",number=10)


In [ ]:
timeit.timeit('set([get_alias("test") for n in range(5)])', "from __main__ import get_alias",number=6)

In [ ]:
timeit.timeit('set([get_alias("test") for n in range(5)])', "from __main__ import get_alias",number=6)

In [ ]:
#without cache
timeit.timeit('set([get_alias("test") for n in range(5)])', "from __main__ import get_alias",number=6)